# Example of loading and processing raw EEM data

* Required meta data columns in CSV/excel
* Loading EEMs
* Process them, including blank subtraction
* Example with different metadata and no blank subtraction or Raman normalization
* Create function to show porcessing steps
    - How to store the order this was conducted in?


In [1]:
import pyeem

pyeem version 0.1


In [2]:
# for development - remove when example notebook is complete
def reload_pyeem():
    import importlib
    importlib.reload(pyeem.data_process)
    importlib.reload(pyeem)
    return
reload_pyeem()

pyeem version 0.1


### Step 1 - generate a pandas dataframe with the required meta data

In [3]:
meta_data = pyeem.load_eem_meta_data('example_data\Description_Example.xlsx')

- The columns 'File_Name' and 'Folder' are required, they point to the data files containing the eems to be loaded.
- 'Blank' and 'Raman_Area' are required if `pyeem.blank_subtract` and `pyeem.raman_normalize` functions will be used.
- All other columns are optional, as many columns as needed to describe the data may be included.

In [4]:
meta_data

,File_Name,Folder,Blank,Raman_Area,Desc,Type
0,ZEF0119_Cig_5ugmL,20180227,20180227_BCycHex0p5sec,1146.3,Cigarette 5 µg/mL,Sample
1,ZEF0132_Diesel_5ugmL,20180227,20180227_BCycHex0p5sec,1146.3,Diesel 5 µg/mL,Sample
2,ZEF00134_Wood_5ugmL,20180227,20180227_BCycHex0p5sec,1146.3,Wood Smoke 5 µg/mL,Sample
3,20180227_BCycHex0p5sec,20180227,20180227_BCycHex0p5sec,1146.3,2018-01-27 Cyclohexane Blank,Blank


### Step 2 - Initialize a h5 file and save the meta data in it.

In [5]:
pyeem.init_h5_database("EEM_data.h5", meta_data, overwrite=True)

overwriting EEM_data.h5


In [8]:
pyeem.load_eems("EEM_data.h5", 'example_data/')

EEM data collection complete, final data shape (Sample x Ex x Em): (4, 250, 151)
Dataset saved: raw_eems ... Shape =  (4, 250, 151)
Dataset saved: eems ... Shape =  (4, 250, 151)
Dataset saved: raw_ex ... Shape =  (4, 151)
Dataset saved: ex ... Shape =  (4, 151)
Dataset saved: raw_em ... Shape =  (4, 250)
Dataset saved: em ... Shape =  (4, 250)


In [9]:
import h5py
with h5py.File("EEM_Data.h5", 'r') as f:
    
#     print('The following data is available in', raw_data_name+'.hdf5')
#     print('-' * 60)
#     print("Key:".ljust(20,' ')) #,"Shape:".ljust(18,' '),"Type:".ljust(10,' '))
#     print('-' * 60)
    for key in f.keys():
        data = f[key]
        print(key.ljust(20,' '))#, str(data.shape).ljust(18,' '), str(data.dtype).ljust(10,' '))

eems                
em                  
ex                  
meta                
raw_eems            
raw_em              
raw_ex              


In [5]:
# step 4 - use the data processing fucntions for all operations
# TO DO - create some sort of log for this porcess...

In [6]:
# finally - display contour plots of the processing steps 
# (for now just code this in the notebook, maybe add a finction later)